In [152]:
import numpy as np
import pandas   as pd
import matplotlib.pyplot as plt
import networkx as nx
import os

### Creiamo un unico dataframe che contiene tutto
Labels(OK)  Degree(OK)	Betweenness(OK)	Voltage	Latitude(OK) Longitude(OK) Old_Labels(OK)

In [153]:
# Load info from original file

with open("./vertices_eu_power_160718.csv",'r') as f:
    
    columns = f.readline().split(',')[0:6]
    df = pd.DataFrame(columns=columns[0:6])
    rows = f.readlines()
    for row in rows:
        df.loc[len(df)] = row.split(',')[0:6]

df["v_id"] = df["v_id"].astype(int)
df.set_index("v_id",inplace=True) # These are the "old" indices
df

,lon,lat,typ,voltage,frequency
v_id,,,,,
1,9.52257601112577,52.3604090734199,substation,220000;110000,50
2,9.11321011483187,52.5438533448625,substation,220000,50
3,9.38974515063535,52.0263130791669,substation,380000;110000,50
4,9.12526574244048,52.5382641233458,substation,380000;220000,50
5,10.366274939217,52.2846467643429,substation,380000;220000,50
...,...,...,...,...,...
1475,24.6883684107371,59.2624545066974,auxillary_T_node,330000,50
1476,6.83264560931642,43.5938493910056,auxillary_T_node,400000,
1477,4.94874901128534,44.9985986181567,auxillary_T_node,225000,50


In [154]:
# Given that some stations have multiple power lines, this function takes the average
def avg_voltage(volt_list):
    volt = []
    c=0
    for item in volt_list:
        print(c)
        c=c+1
        x = str(item).split(";")
        x = [float(y) for y in x] if len(x) > 1 else float(x[0] or 0)
        x = sum(x) / len(x) if isinstance(x, list) else x
        volt.append(x)
    return volt

# grid["voltage"] =  avg_voltage(grid["voltage"]) 

df["voltage"]    = avg_voltage(df["voltage"]) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [157]:
df

,lon,lat,typ,voltage,frequency
v_id,,,,,
0,9.52257601112577,52.3604090734199,substation,165000.0,50
1,9.11321011483187,52.5438533448625,substation,220000.0,50
2,9.38974515063535,52.0263130791669,substation,245000.0,50
3,9.12526574244048,52.5382641233458,substation,300000.0,50
4,10.366274939217,52.2846467643429,substation,300000.0,50
...,...,...,...,...,...
1462,24.6883684107371,59.2624545066974,auxillary_T_node,330000.0,50
1463,6.83264560931642,43.5938493910056,auxillary_T_node,400000.0,
1464,4.94874901128534,44.9985986181567,auxillary_T_node,225000.0,50


In [156]:
labels        = pd.read_csv("./old_new_labels.txt", header=None, sep=" ", names=["old_labels", "new_labels"])

# Prima selezioniamo i nodi connessi coi vecchi labels salvati in old_labels, infatti grid ha 1467, potrebbero esserci nodi disconnessi come in US power grid (non è questo il caso)
df = df.loc[labels["old_labels"]]

# Riportiamo tutto ai nuovi indici
df.rename(index={k:v for k,v in zip(labels["old_labels"],labels["new_labels"])}, inplace=True) # Relabelling old to new indices


In [160]:
df.rename(columns={"lon":"Longitude","lat":"Latitude","voltage":"Voltage","typ":"Type"},inplace=True)

# [ "Degree","Betweenness","Voltage","Latitude","Longitude","Old_Labels"]

In [179]:
df

,Longitude,Latitude,Type,Voltage,frequency,Degree,Betweennwss,Old_Labels
v_id,,,,,,,,
0,9.52257601112577,52.3604090734199,substation,165000.0,50,1,0.000000,1
1,9.11321011483187,52.5438533448625,substation,220000.0,50,5,0.006964,2
2,9.38974515063535,52.0263130791669,substation,245000.0,50,5,0.023936,3
3,9.12526574244048,52.5382641233458,substation,300000.0,50,5,0.027088,4
4,10.366274939217,52.2846467643429,substation,300000.0,50,8,0.018333,5
...,...,...,...,...,...,...,...,...
1462,24.6883684107371,59.2624545066974,auxillary_T_node,330000.0,50,3,0.000004,1475
1463,6.83264560931642,43.5938493910056,auxillary_T_node,400000.0,,3,0.000007,1476
1464,4.94874901128534,44.9985986181567,auxillary_T_node,225000.0,50,3,0.000970,1477


In [2]:
df_PowerGrid  = pd.read_csv('Data/EU_PowerGrid/node_coordinates.nodelist', delimiter=' ',names=["Label","Latitude","Longitude"],header=None)
degree        = pd.read_csv("Data/EU_PowerGrid/gridkit_north_america.nodelist", delimiter=' ',names=["Label","Degree","node_betweenness"],header=0)
labels        = pd.read_csv("./Data/EU_PowerGrid/old_new_labels.txt", header=None, sep=" ", names=["old_labels", "new_labels"])

df_Nodes = pd.DataFrame(columns=[ "Degree","Betweenness","Voltage","Latitude","Longitude","Old_Labels"], 
                            index = df_PowerGrid["Label"])
df_Nodes["Latitude"]  = df_PowerGrid["Latitude"]
df_Nodes["Longitude"] = df_PowerGrid["Longitude"]
df_Nodes["Degree"] = degree["Degree"]
df_Nodes["Betweenness"] = degree["node_betweenness"]
df_Nodes["Old_Labels"] = labels["old_labels"]

In [97]:
# Qui prepariamo la colonna del voltaggio
grid   = pd.read_csv('Data/PowerGrid/gridkit_north_america-highvoltage-vertices_original.csv',delimiter=',',index_col="v_id")

# Prima selezioniamo i nodi connessi coi vecchi labels salvati in old_labels, infatti grid ha 16174, mentre df_Nodes (i nodi connessi) sono 16167, quindi escludiamo i 7 mancanti
grid = grid.loc[labels["old_labels"]]

# Riportiamo tutto ai nuovi indici
grid.rename({k:v for k,v in zip(labels["old_labels"],labels["new_labels"])}, inplace=True) # Relabelling old to new indices

# Given that some stations have multiple power lines, this function takes the average
def avg_voltage(volt_list):
    volt = []
    for item in volt_list:
        x = str(item).split(";")
        x = [float(y) for y in x] if len(x) > 1 else float(x[0])
        x = sum(x) / len(x) if isinstance(x, list) else x
        volt.append(x)
    return volt

# grid["voltage"] =  avg_voltage(grid["voltage"]) 

df_Nodes["Voltage"]    = avg_voltage(grid["voltage"]) 
df_Nodes["Frequency"]  = grid["frequency"]

In [107]:
# Qui ci occupiamo del tipo di griglia
grid["typ"] 
unique_list = list(set(grid["typ"])) # Elementi unici
my_dict = {'sub_station': 'substation'} # Relabelling

type_grid = [my_dict.get(item, item) for item in grid["typ"]]
df_Nodes["Type"] = type_grid 

In [117]:
df_Nodes_Final = df_Nodes.reindex(columns=['Degree', 'Betweenness', 'Voltage', 'Type', 'Frequency','Latitude', 'Longitude',   'Old_Labels'])

In [119]:
df_Nodes_Final.to_csv("./Data/PowerGrid/vertices_grid.csv", sep=' ')


### Find network metrics 

In [143]:
G = nx.read_edgelist("./eu-powergrid.el")  # New labels

In [144]:
b = nx.betweenness_centrality(G)  # About 9 min to compute for 16k nodes
d= {int(key):value for key,value in b.items()}

In [145]:
# plt.hist(d.values(),30,range=(0.001,0.02));
dd =G.degree()
dd = {int(key):value for key,value in dict(dd).items()}
# nodes = pd.DataFrame({'node_degree':dd,'node_betweenness':d},).sort_index()

In [167]:
df["Degree"] = dd
df["Betweennwss"] = d
df["Old_Labels"]  = labels["old_labels"]


In [177]:
df.to_csv("./vertices_eu_grid.csv",sep=' ')

In [176]:
# Save the nodelist
df["Degree"].to_csv("eu-powergrid.nodelist",sep=' ')

Leggi edgelist originale e cambia i labels

In [37]:
G = nx.read_edgelist("./eu-powergrid_original.el")

In [38]:
# Rename labels from 0 to N-1
old_labels = G.nodes()
GG = nx.convert_node_labels_to_integers(G,ordering="default")  # Inherit ordering from G.nodes() (default)

In [39]:
# Salva la edgelist con i labels modificati
with open('eu-powergrid.el', 'w') as f:
    for a,b in GG.edges():
        f.write(str(a) + " " + str(b) + "\n")

In [5]:
# Salva la corrispondenza old_labels - new_labels
with open('old_new_labels.txt', 'w') as f:
    for a,b in zip(old_labels,GG.nodes()):
        f.write(str(a) + " " + str(b) + "\n")

In [5]:
# Salva la nodelist del grafo riordinato coi nuovi labels (degree)

degree_sequence = list((d for n, d in GG.degree()))
nodelist = [int(string) for string in [n for n,d in GG.degree()]] 

with open('gridkit_north_america-highvoltage-vertices.nodelist', 'w') as f:
    for a,b in zip(nodelist,degree_sequence):
        f.write(str(a) + " " + str(b) + "\n")

In [6]:
# Salva la nodelist del grafo originale
degree_sequence_old = list((d for n, d in G.degree()))
nodelist_old        = [int(string) for string in [n for n,d in G.degree()]]
with open('gridkit_north_america-highvoltage-vertices_origin.nodelist', 'w') as f:
    for a,b in zip(nodelist_old, degree_sequence_old):
        f.write(str(a) + " " + str(b) + "\n")

### Estrai latitudine e longitudine dal file originale e salva i risultati usando i nuovi_label
Prendiamo la nodelist con i nuovi label, per ogni elemento troviamo il corrispondente nel file **old_new_label**, infine trovato questo corrispondente lo cerchiamo nel dataset originale ed estraiamo latitudine e longitudine, infine salviamo tutto in una nuova nodelist

In [84]:
file_grid     = "./Data/PowerGrid/gridkit_north_america-highvoltage-vertices_original.csv"
file_nodelist = "./Data/PowerGrid/gridkit_north_america.nodelist" # Qui ci sono i nuovi label
file_label    = "./Data/PowerGrid/old_new_labels.txt"
nodeslist     = pd.read_csv(file_nodelist, sep=" ", names=["new_labels", "degree"])
labels        = pd.read_csv(file_label, header=None, sep=" ", names=["old_labels", "new_labels"])
orig_data     = pd.read_csv(file_grid)

In [115]:
# Salva una nuova nodelist coi nuovi labels e latitudine e longitudine
with open('node_coordinates.nodelist', 'w') as f:

    for old_lab, new_lab in zip(labels["old_labels"], labels["new_labels"]):
        ind = list(orig_data["v_id"]).index(old_lab)    # Troviamo la riga corrispondente nel dataset originale
        lat, lon = orig_data.iloc[ind][["lat","lon"]]   # Estraiamo latitudine e longitudine
        f.write( str(new_lab) + " " + str(lat) + " " + str(lon) + "\n")

### Aggiungiamo una colonna con in nuovi label nel file 

In [4]:
# LEGGIAMO I DATI
file_grid     = "./Data/PowerGrid/gridkit_north_america-highvoltage-vertices_original.csv"
file_nodelist = "./Data/PowerGrid/gridkit_north_america.nodelist" # Qui ci sono i nuovi label
file_label    = "./Data/PowerGrid/old_new_labels.txt"
nodeslist     = pd.read_csv(file_nodelist, sep=" ", names=["new_labels", "degree"])
labels        = pd.read_csv(file_label, header=None, sep=" ", names=["old_labels", "new_labels"])
orig_data     = pd.read_csv(file_grid)

In [95]:
label_dict       = dict(zip(labels.old_labels, labels.new_labels))
relabeled_vector = [int(label_dict[item]) if item in label_dict.keys() else np.nan for item in orig_data.v_id ]

In [99]:
orig_data.head()

,v_id,lon,lat,typ,voltage,frequency,name,operator,ref,wkt_srid_4326,new_v_id
0,23470,-90.040427,38.738671,joint,345000,60,NaN,NaN,NaN,SRID=4326;POINT(-90.0404267803716 38.738670960...,7009.0
1,16854,-93.895894,49.064320,substation,NaN,NaN,NaN,NaN,NaN,SRID=4326;POINT(-93.8958936452349 49.064320449...,15348.0
2,25563,-75.159609,40.707010,joint,230000;69000,NaN,NaN,NaN,NaN,SRID=4326;POINT(-75.15960889129 40.7070100889546),3003.0
3,6044,-78.839873,42.789666,sub_station,NaN,NaN,NaN,NaN,NaN,SRID=4326;POINT(-78.8398726969668 42.789666486...,7643.0
4,8477,-94.638969,47.379042,substation,NaN,NaN,NaN,NaN,NaN,SRID=4326;POINT(-94.6389691818178 47.379042346...,11089.0


In [101]:
orig_data.to_csv("./Data/PowerGrid/gridkit_north_america-highvoltage-vertices_original.csv", sep=' ')

In [69]:
label_dict = dict(zip(labels.old_labels, labels.new_labels))
# relabeled_vector = np.vectorize(label_dict.get)([3,13322,8105])   # ([3,13322,8105]) = (old_labels)
relabeled_vector = np.vectorize(label_dict.get)(orig_data.v_id[0:1000])
relabeled_vector

array([ 7009, 15348,  3003,  7643, 11089,  1228,  2555,  4099, 13529,
        9993,  6100,  7750, 16147, 12908,  3831,   171,  4636, 11661,
       10121,  3258,  8845,   881,  3549,  7440, 14097,  8318,  3333,
        1216, 15233,  3630,  4514, 16088, 12991,  4529, 14545,  2914,
       11177,  6253,  3744, 15207,  3443,  7128, 11108, 10630, 15364,
       13421, 13161,  5025,  5638, 14676,  8057,  1733,  1687,  1208,
          73,  6244,  6072,  7975,  1870, 15821,  7459,  3994, 12719,
        1674,   750,  1117,  5381,  2315, 14168,  1129, 14261, 10694,
       11767, 11037, 10000,  4538,  6823,  6887, 10876, 12021,  3348,
       12191, 11268,  4533,  5030,  6610,  3547, 14755,  9289,  2620,
       14429,  2164,  9128,  5785,  8995,  6080,  6997,  5847,  1819,
       11334,   594, 11433,  5220, 10057, 14835,  7713,  6602,  5596,
        3070, 13470, 15578,   249, 15383,  2775,  4017,  8447, 13375,
        3021,  2037, 15235, 10214, 14206,   264, 13557,   884, 15152,
        6582, 13592,